In [17]:
import pandas as pd

from datetime import date

from project_huishoudboekje.config import GeneralSettings

In [2]:
df = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions.xlsx').sort_values(by='DATE', ascending=False)

In [3]:
def prepare_data(df):

    df_analysis = df.loc[df.ANALYSE_IND == 1]

    df_analysis['YEAR_MONTH'] = pd.DatetimeIndex(df_analysis.DATE).strftime("%Y-%m")

    df_analysis['AMOUNT_NW'] = df_analysis.apply(
        lambda x: -x['AMOUNT'] if ((x['FINANCIAL_TYPE'] == 'credit') & (x['GROUP'] != 'Inkomsten')) | (
                (x['FINANCIAL_TYPE'] == 'debet') & (x['GROUP'] == 'Inkomsten')) else x['AMOUNT'], axis=1)

    df_analysis['INCOME_IND'] = df['GROUP'].apply(lambda x: x if x == 'Inkomsten' else 'Uitgaven')

    return df_analysis

In [4]:
df_analysis = prepare_data(df)

C:\Users\robsc\AppData\Local\Temp\ipykernel_5828\2280867606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['YEAR_MONTH'] = pd.DatetimeIndex(df_analysis.DATE).strftime("%Y-%m")
C:\Users\robsc\AppData\Local\Temp\ipykernel_5828\2280867606.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['AMOUNT_NW'] = df_analysis.apply(
C:\Users\robsc\AppData\Local\Temp\ipykernel_5828\2280867606.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [5]:
df_sel = df_analysis.groupby('CATEGORY')['AMOUNT'].sum()

In [6]:
df_sel

CATEGORY
Aansprakelijkheidsverzekering       11.19
Amazon Prime                        17.43
Apple iCloud                         6.93
Autoverzekering                     54.74
Badkamer                         19255.81
                                   ...   
Water                               38.91
Waterschapsbelasting               175.55
Wegenbelasting                     236.00
ZOA                                 30.00
Zorgverzekering                   1299.79
Name: AMOUNT, Length: 62, dtype: float64

In [7]:
df_budget = pd.read_excel(GeneralSettings.project_path / 'data/budget.xlsx')

In [8]:
df_budget_tot = df_budget.groupby('CATEGORY')['BUDGET'].sum()

In [13]:
df_tot = pd.concat([df_budget_tot, df_sel], axis=1)

In [15]:
df_tot['DELTA'] = df_tot['BUDGET'] - df_tot['AMOUNT']
df_tot['RATIO'] = df_tot['AMOUNT'] / df_tot['BUDGET']

In [16]:
df_tot

,BUDGET,AMOUNT,DELTA,RATIO
CATEGORY,,,,
Aansprakelijkheidsverzekering,4682,11.19,4670.81,0.002390
Amazon Prime,5800,17.43,5782.57,0.003005
Apple iCloud,5509,6.93,5502.07,0.001258
Autoverzekering,4913,54.74,4858.26,0.011142
Badkamer,5194,19255.81,-14061.81,3.707318
...,...,...,...,...
Water,6905,38.91,6866.09,0.005635
Waterschapsbelasting,6412,175.55,6236.45,0.027378
Wegenbelasting,6014,236.00,5778.00,0.039242


In [21]:
date.today().replace(day=1)

datetime.date(2022, 8, 1)

In [24]:
df_sel = df_analysis[df_analysis['DATE'] < date.today().replace(day=1).strftime('%Y-%m-%d')]

In [25]:
df_budget['DATE'] = pd.to_datetime(df_budget['YEAR_MONTH'], format='%Y-%m')

In [27]:
df_budget_sel = df_budget[df_budget['DATE'] < date.today().replace(day=1).strftime('%Y-%m-%d')]

In [36]:
def create_table_output(df_analysis, df_budget, view_value):
    
    if view_value == 'YMD':
        
        ref_date = date.today().replace(day=1)
        
        df_analysis = df_analysis[df_analysis['DATE'] < ref_date.strftime('%Y-%m-%d')]
        df_budget = df_budget[df_budget['DATE'] < ref_date.strftime('%Y-%m-%d')]
        
    df_analysis_tot = df_analysis.groupby('CATEGORY')['AMOUNT'].sum()
    df_budget_tot = df_budget.groupby('CATEGORY')['BUDGET'].sum()
        
    df_tot = pd.concat([df_budget_tot, df_analysis_tot], axis=1).fillna(0)
    
    df_tot['DELTA'] = df_tot['BUDGET'] - df_tot['AMOUNT']
    df_tot['RATIO'] = df_tot['AMOUNT'] / df_tot['BUDGET']
    
    return df_tot.reset_index().sort_values(by='AMOUNT', ascending=False)

In [37]:
df_final = create_table_output(df_analysis, df_budget, 'Total')

In [38]:
df_final

,CATEGORY,BUDGET,AMOUNT,DELTA,RATIO
48,Salaris Rob,4954,27541.30,-22587.30,5.559407
4,Badkamer,5194,19255.81,-14061.81,3.707318
26,Hypotheek,6283,8827.21,-2544.21,1.404936
12,Bruiloft,5498,8460.65,-2962.65,1.538860
41,Overige inkomsten,5121,6531.00,-1410.00,1.275337
...,...,...,...,...,...
2,Apple iCloud,5509,6.93,5502.07,0.001258
32,Massage,7069,0.00,7069.00,0.000000
21,Fysiotherapie,7280,0.00,7280.00,0.000000
16,Drieluik,5481,0.00,5481.00,0.000000
